In [46]:
from nilearn import plotting
from nilearn.image import resample_img, mean_img
from nilearn import datasets
import nibabel as nib
from nipype.interfaces import fsl
from nipype.interfaces.ants import N4BiasFieldCorrection, RegistrationSynQuick
from lib.zscore_norm import zscore_normalize
from utils import applyxfm4d


atlas_folder = "/data/haoyuezhang/data/vascular_territory_template/"
data_test_folder = '/data/haoyuezhang/data/vascular_territory_template/testcase/'
atl_terr_path = atlas_folder+'mni_vascular_territories.nii.gz'
atl_flair_path = atlas_folder + 'caa_flair_in_mni_template_smooth_brain_intres.nii.gz'

In [ ]:
pwi_path = data_test_folder+'540016/PWI.nii.gz'
pwi = nib.load(pwi_path)
print(pwi.shape)

In [22]:
flair_path = data_test_folder+'FLAIR.nii.gz'
flair = nib.load(flair_path)
print(flair.shape)

(232, 256, 26)


In [ ]:
atl_terr = nib.load(atl_terr_path)
atl_flair = nib.load(atl_flair_path)
print(atl_flair.shape)

In [23]:
## manual registration starts here

reorient = fsl.utils.Reorient2Std()
reorient.inputs.in_file = flair_path
reorient.inputs.out_file = flair_path
res = reorient.run() 

In [31]:
# robust fov to remove neck and lower head automatically

rf = fsl.utils.RobustFOV()
rf.inputs.in_file = data_test_folder+'FLAIR.nii.gz'
rf.inputs.out_roi = data_test_folder+'FLAIR_RF.nii.gz'
rf.cmdline
rf.run()

200917-19:45:41,970 nipype.interface INFO:
	 stdout 2020-09-17T19:45:41.970670:Final FOV is: 
200917-19:45:41,971 nipype.interface INFO:
	 stdout 2020-09-17T19:45:41.970670:0.000000 232.000000 0.000000 256.000000 0.000000 25.000000 
200917-19:45:41,972 nipype.interface INFO:
	 stdout 2020-09-17T19:45:41.970670:


In [32]:
# two run experiment
btr1 = fsl.BET()
btr1.inputs.in_file = data_test_folder+'FLAIR_RF.nii.gz'
btr1.inputs.robust = True
btr1.inputs.frac = 0.2
btr1.inputs.out_file = data_test_folder+'FLAIR_RF.nii.gz'
res = btr1.run()

In [33]:
n4 = N4BiasFieldCorrection()
n4.inputs.dimension = 3
n4.inputs.input_image = data_test_folder+'FLAIR_RF.nii.gz'
n4.inputs.bspline_fitting_distance = 300
n4.inputs.shrink_factor = 3
n4.inputs.n_iterations = [50,50,30,20]
n4.inputs.output_image = data_test_folder+'FLAIR_RF.nii.gz'
res = n4.run()

In [83]:
flt = fsl.FLIRT(cost_func='leastsq', interp='trilinear',
                searchr_x=[-180, 180], searchr_y=[-180, 180], searchr_z=[-180,180],dof=12)
flt.inputs.in_file = data_test_folder+'FLAIR_RF.nii.gz'
flt.inputs.reference = atl_flair_path
flt.inputs.out_file = data_test_folder+'FLAIR_first_run_r.nii.gz'
flt.inputs.out_matrix_file = data_test_folder+'FLAIR_r_transform.mat'
res = flt.run()

In [56]:
btr2 = fsl.BET()
btr2.inputs.in_file = data_test_folder+'FLAIR_first_run_r.nii.gz'
btr2.inputs.robust = True
btr2.inputs.frac = 0.2
btr2.inputs.mask = True
btr2.inputs.out_file = data_test_folder+'FLAIR_r.nii.gz'
res = btr2.run()

In [57]:
# z score normalization
FLAIR_path = data_test_folder+'FLAIR_r.nii.gz'
FLAIR_final = nib.load(FLAIR_path)
FLAIR_mask_path = data_test_folder+ 'FLAIR_r_mask.nii.gz'
mask = nib.load(FLAIR_mask_path)
FLAIR_norm = zscore_normalize(FLAIR_final, mask)
nib.save(FLAIR_norm, FLAIR_path)

In [58]:
import os
# register with pwi

# automatic reorient to MNI direction
reorient = fsl.utils.Reorient2Std()
reorient.inputs.in_file = pwi_path
reorient.inputs.out_file = data_test_folder+'PWI.nii.gz'
res = reorient.run() 
                


0

In [68]:
btr1 = fsl.BET()
btr1.inputs.in_file = data_test_folder+'PWI.nii.gz'
btr1.inputs.frac = 0.2
btr1.inputs.out_file = data_test_folder+'PWI.nii.gz'
btr1.inputs.functional = True
res = btr1.run()

In [84]:
os.chdir(data_test_folder)
# either use my customized applyxf4d wrapper for perfusion data or directly run this CMD
applyXFM_cmd = "applyxfm4D PWI.nii.gz FLAIR_first_run_r.nii.gz PWI_r.nii.gz FLAIR_r_transform.mat -singlematrix"
os.system(applyXFM_cmd)

0

In [ ]:
btr2 = fsl.BET()
btr2.inputs.in_file = data_test_folder+'PWI_r.nii.gz'
btr2.inputs.functional = True
btr2.inputs.frac = 0.2
btr2.inputs.mask = True
btr2.inputs.out_file = data_test_folder+'PWI_r.nii.gz'
res = btr2.run()

In [79]:
flt = fsl.FLIRT(cost_func='mutualinfo', interp='trilinear',
                searchr_x=[-180, 180], searchr_y=[-180, 180], searchr_z=[-180,180],dof=12)
flt.inputs.in_file = data_test_folder+'PWI.nii.gz'
flt.inputs.reference = data_test_folder+'FLAIR_r.nii.gz'
flt.inputs.out_file = data_test_folder+'PWI_r.nii.gz'
flt.inputs.in_matrix_file = data_test_folder+'FLAIR_r_transform.mat'
flt.inputs.apply_xfm = True
res = flt.run()

200918-12:27:38,71 nipype.interface INFO:
	 stderr 2020-09-18T12:27:38.071023:Image Exception : #75 :: 3D only method called by higher-dimensional volume.
200918-12:27:38,75 nipype.interface INFO:
	 stderr 2020-09-18T12:27:38.075553:3D only method called by higher-dimensional volume.


In [61]:
am = fsl.maths.ApplyMask()
am.inputs.in_file = data_test_folder+'PWI_r.nii.gz'
am.inputs.mask_file = data_test_folder+'FLAIR_r_mask.nii.gz'
am.inputs.out_file = data_test_folder+'PWI_r_m.nii.gz'
res = am.run()

In [78]:
flt = fsl.FLIRT(cost_func='mutualinfo', interp='trilinear',
                searchr_x=[-180, 180], searchr_y=[-180, 180], searchr_z=[-180,180],dof=12)
flt.inputs.in_file = data_test_folder+'ADC.nii.gz'
flt.inputs.in_matrix_file = data_test_folder+'FLAIR_r_transform.mat'
flt.inputs.out_file = data_test_folder+'ADC_r.nii.gz'
flt.inputs.reference = data_test_folder+'FLAIR_r.nii.gz'
#flt.inputs.apply_xfm = True
result = flt.run() 

In [92]:
# use ANTs to register
import copy, pprint
from nipype.interfaces.ants import Registration

reg = Registration()
reg.inputs.fixed_image = atl_flair_path
reg.inputs.moving_image = data_test_folder+'FLAIR_RF.nii.gz'
reg.inputs.output_transform_prefix = data_test_folder+'FLAIR_r_transform.mat'
reg.inputs.transforms = ['Affine']
reg.inputs.transform_parameters = [(2.0,), (0.25, 3.0, 0.0)]
reg.inputs.number_of_iterations = [[1500, 200], [100, 50, 30]]
reg.inputs.dimension = 3
reg.inputs.write_composite_transform = True
reg.inputs.collapse_output_transforms = False
reg.inputs.initialize_transforms_per_stage = False
reg.inputs.metric = ['Mattes', ['Mattes', 'CC']]
reg.inputs.metric_weight = [1, [.5,.5]] # Default (value ignored currently by ANTs)
reg.inputs.radius_or_number_of_bins = [32]*2
reg.inputs.sampling_strategy = ['Random', None]
reg.inputs.sampling_percentage = [0.05, None]
reg.inputs.convergence_threshold = [1.e-8, 1.e-9]
reg.inputs.convergence_window_size = [20]*2
reg.inputs.smoothing_sigmas = [[1,0], [2,1,0]]
reg.inputs.sigma_units = ['vox'] * 2
reg.inputs.shrink_factors = [[2,1], [3,2,1]]
reg.inputs.use_estimate_learning_rate_once = [True, True]
reg.inputs.use_histogram_matching = [True, True] # This is the default
reg.inputs.output_warped_image = data_test_folder+'ANTS_FLAIR_r.nii.gz'


reg.run()


In [100]:
from nipype.interfaces.ants import ApplyTransforms

at = ApplyTransforms()
at.inputs.dimension = 3
at.inputs.input_image_type = 3
at.inputs.input_image = data_test_folder+'PWI.nii.gz'
at.inputs.reference_image = data_test_folder+'ANTS_FLAIR_r.nii.gz'
at.inputs.output_image = data_test_folder+'PWI_r.nii.gz'
at.inputs.interpolation = 'Linear'
at.inputs.default_value = 0
at.inputs.transforms = data_test_folder+'FLAIR_r_transform.matComposite.h5'
at.run()